In [ ]:
import pandas as pd
import geopandas as gpd
from siuba import *

In [ ]:
import zipfile

In [ ]:
! pip install pygris

In [ ]:
import _utils
import importlib
importlib.reload(_utils)

In [ ]:
from _utils import GCS_PATH
from calitp_data_analysis import geography_utils

# "What if VMT decreased by 25% per the CARB target, and all those trips were on (existing) transit instead?"

## VMT is a spatial phenomenon, our analysis should be spatial too

* Start with "big data" weekday residential VMT per Census tract via Replica
* Per target, future VMT should be 25% less
* Assume tripmaking remains constant, and that transit entirely replaces that VMT
    * optional: find tracts with no transit service, hold their VMT constant and redistribute missed target among remaining tracts (30% reduction instead of 25% perhaps?)
    
## From reduced VMT to transit trips

* Replica gives transit trip lengths but it may not be reliable ("good for auto, less so for transit")
    * It's generally showing the median transit trip as longer than the median auto trip, which seems questionable
    * We have plenty of good spatial data on transit service _provision_, but not ridership (generally agency-level only)
    * May need to refer to research/default to a fixed "median transit trip" length based on population density
* Regardless, get a rough estimate by dividing reduced VMT in each tract by median transit trip distance, or median auto trip if transit data not available

In [ ]:
gdf = gpd.read_parquet(f'{GCS_PATH}outputs/new_trips_with_uza.parquet')

## This gdf has model results for whole state, filter then map...

In [ ]:
#  filtered test
gdf = gdf >> filter(_.NAME10.str.contains('Seaside'))

## Mapping...

In [ ]:
# gdf.explore(column = 'total_mi_transit', scheme = 'NaturalBreaks')

In [ ]:
gdf.explore(column = 'total_mi_auto', scheme = 'NaturalBreaks')

## New transit trips

In [ ]:
def map_per_capita(gdf):
    gdf = gdf >> filter(_.total_pop != 0) # remove tracts where nobody lives
    return gdf.explore(column = 'new_trips_per_capita', scheme = 'NaturalBreaks')

In [ ]:
map_per_capita(gdf)

In [ ]:
gdf.explore(column = 'projected_new_transit_trips', scheme = 'NaturalBreaks')

In [ ]:
gdf.projected_new_transit_trips.sum()

## Next Steps

* caveat: other strategies (land use, active modes...)
* caveat: induced travel
* stratify into "good transit, not riding", "bad transit"
* LODES o/d data? Replica? -> Conveyal transit o/d find that "good transit but not riding it"
   * find what doesn't show up in aggregate accessibility...
* https://walker-data.com/pygris/